In [ ]:
# Mounting drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
# import cv2
# import os
# vid_path = "/content/gdrive/My Drive/291 Bionic Vision Project/Asa_Video/stim1_vid(done)"
# vidcap = cv2.VideoCapture('%s/stim1.mp4' % vid_path)
# os.mkdir("%s/orig" %(vid_path))
# success,image = vidcap.read()
# count = 0
# while success:
#   cv2.imwrite("%s/orig/frame_orig%d.jpg" % (vid_path,count), image)     # save frame as JPEG file      
#   success,image = vidcap.read()
#   print('Read a new frame: ', success)
#   count += 1

# mask RCNN

In [ ]:
# !pip install tensorflow==1.13.1
%tensorflow_version 1.x
import tensorflow as tf

print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
# !git clone https://github.com/matterport/Mask_RCNN.git
import os
os.chdir('/content/gdrive/My Drive/Mask_RCNN/samples/coco')
!ls

coco.py		    inspect_model.ipynb    Mask_RCNN	seg_1.png
inspect_data.ipynb  inspect_weights.ipynb  __pycache__


In [ ]:
import os
import glob
import sys

import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.axes as ax
import json
import skimage.io
from skimage import measure
from scipy.io import loadmat
from scipy.spatial import distance
import cv2
from PIL import Image

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.utils import Dataset

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Path to Shapes trained weights
SHAPES_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_shapes.h5")

Using TensorFlow backend.


In [ ]:
import coco
config = coco.CocoConfig()
COCO_DIR = "/content/coco" 

# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
# config.display()


# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

# Set weights file path
if config.NAME == "shapes":
    weights_path = SHAPES_MODEL_PATH
elif config.NAME == "coco":
    weights_path = COCO_MODEL_PATH
# Or, uncomment to load the last model you trained
# weights_path = model.find_last()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

class_names =  ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.
Loading weights  /content/gdrive/My Drive/Mask_RCNN/mask_rcnn_coco.h5


# Deep Gaze II

In [ ]:
import os
os.chdir('/content/gdrive/My Drive')
!ls

In [ ]:
!git clone https://github.com/mpatacchiola/deepgaze.git

In [ ]:
os.chdir("/content/gdrive/My Drive/deepgaze")
!ls

debug.log  deepgaze  doc  etc  examples  LICENSE  README.md  setup.py


In [ ]:
#!/usr/bin/env python

# The MIT License (MIT)
# Copyright (c) 2017 Massimiliano Patacchiola
# https://mpatacchiola.github.io
# https://mpatacchiola.github.io/blog/
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
# MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY
# CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE
# SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# In this example the FASA algorithm is used in order to process some images.
# The original image and the saliency version are showed for comparison.

import numpy as np
import cv2
from timeit import default_timer as timer
from deepgaze.saliency_map import FasaSaliencyMapping 
import os
import glob
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.axes as ax
import json
import skimage.io
from scipy.io import loadmat
import cv2
from PIL import Image

# def main():

#     image_1 = cv2.imread("./horse.jpg")
#     image_2 = cv2.imread("./car.jpg")
#     image_3 = cv2.imread("./plane.jpg")
#     image_4 = cv2.imread("./pear.jpg")

#     # for each image the same operations are repeated
#     my_map = FasaSaliencyMapping(image_1.shape[0], image_1.shape[1])  # init the saliency object
#     start = timer()
#     image_salient_1 = my_map.returnMask(image_1, tot_bins=8, format='BGR2LAB')  # get the mask from the original image
#     image_salient_1 = cv2.GaussianBlur(image_salient_1, (3,3), 1)  # applying gaussin blur to make it pretty
#     end = timer()
#     print("--- %s Image 1 tot seconds ---" % (end - start))

#     my_map = FasaSaliencyMapping(image_2.shape[0], image_2.shape[1])
#     start = timer()
#     image_salient_2 = my_map.returnMask(image_2, tot_bins=8, format='BGR2LAB')
#     image_salient_2 = cv2.GaussianBlur(image_salient_2, (3,3), 1)
#     end = timer()
#     print("--- %s Image 2 tot seconds ---" % (end - start))

#     my_map = FasaSaliencyMapping(image_3.shape[0], image_3.shape[1])
#     start = timer()
#     image_salient_3 = my_map.returnMask(image_3, tot_bins=8, format='BGR2LAB')
#     #image_salient_3 = cv2.GaussianBlur(image_salient_3, (3,3), 1)
#     end = timer()
#     print("--- %s Image 3 tot seconds ---" % (end - start))

#     my_map = FasaSaliencyMapping(image_4.shape[0], image_4.shape[1])
#     start = timer()
#     image_salient_4 = my_map.returnMask(image_4, tot_bins=8, format='BGR2LAB')
#     image_salient_4 = cv2.GaussianBlur(image_salient_4, (3,3), 1)
#     end = timer()
#     print("--- %s Image 4 tot seconds ---" % (end - start))

#     # Creating stack of images and showing them on screen
#     original_images_stack = np.hstack((image_1, image_2, image_3, image_4))
#     saliency_images_stack = np.hstack((image_salient_1, image_salient_2, image_salient_3, image_salient_4))
#     saliency_images_stack = np.dstack((saliency_images_stack,saliency_images_stack,saliency_images_stack))
#     cv2.imshow("Original-Saliency", np.vstack((original_images_stack, saliency_images_stack)))

#     while True:
#         if cv2.waitKey(33) == ord('q'):
#             cv2.destroyAllWindows()
#             break


# if __name__ == "__main__":
#     main()

In [ ]:
from skimage import feature
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import euclidean_distances
import glob

### functions

In [ ]:
def sal_mask(image):
  '''
  PARAMETERS:
    image: input image [h, w, 3]
  
  RETURN:
    origs: original images [h, w, 3, #images]
    origs_gray: original images in grayscale [h, w, #images]
    saliens: deepgaze saliency map of images [h, w, #images]
    masks: combined object masks of images [h, w, #images]
  '''
  h,w,_ = image.shape

  # get masks
  results = model.detect([image])
  r = results[0]
  obj_masks = r["masks"]
  # fig = visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
  #                           class_names, r['scores'])

  # get mask edges
  num_objs = r["masks"].shape[2]
  obj_edges = np.zeros(r["masks"].shape)
  for o in range(num_objs):
    m = r["masks"][:,:,o]
    edges = feature.canny(m, sigma=5).astype("uint8")
    kernel = np.ones((10,10), np.uint8) 
    edges_dilated = cv2.dilate(edges,kernel)
    obj_edges[:,:,o] = edges_dilated

  # # get all object masks into one mask
  # combined_masks = np.sum(r['masks'],2)

  # get saliency map
  my_map = FasaSaliencyMapping(h, w) 
  image_salient = my_map.returnMask(image, tot_bins=8, format='BGR2LAB')  # get the mask from the original image
  image_salient_blr = cv2.GaussianBlur(image_salient, (3,3), 1)

  return image_salient_blr, obj_masks, r["rois"], r["class_ids"], obj_edges #roi top-left and bottom-right y,x


def intersec_area(a, b):  # returns 0 if rectangles don't intersect
    dx = np.min([a[3], b[3]]) - np.max([a[1], b[1]])
    dy = np.mean([a[2], b[2]]) - np.max([a[0], b[0]])
    if (dx>=0) and (dy>=0):
        return dx*dy
    else: 
      return 0

### read original video and set up

In [ ]:
# vid_path = "/content/gdrive/My Drive/291 Bionic Vision Project/Asa_Video/stim1_vid(done)" # ORIGINAL VIDEO PATH FOLDER 
# vid_name = 'stim1.mp4'

# cap = cv2.VideoCapture('%s/%s'% (vid_path, vid_name))

# ret, frame1 = cap.read()

# results = model.detect([frame1])
# rs = results[0]
# # fig = visualize.display_instances(frame1, rs['rois'], rs['masks'], rs['class_ids'], 
#                           # class_names, rs['scores'])


# # set up optical flow 
# prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
# hsv = np.zeros_like(frame1)
# hsv[...,1] = 255

# # saliency observation window
# W = 10
# w_counter = 1
# s, m, roi, ids, obj_edges = sal_mask(frame1)
# s_history = np.copy(s)
# s_history = np.expand_dims(s_history, axis=2)

# roi_history = []
# roi_history.append(roi)

In [ ]:
# fig = visualize.display_instances(frame1, rs['rois'], rs['masks'], rs['class_ids'], 
#                           class_name?s, rs['scores'])

### Segmentation (enhanced by optical flow) + Saliency (smoothied from previous frames and filted out large variance)


In [ ]:
# importing the necessary libraries 
import cv2 
import numpy as np 

# Creating a VideoCapture object to read the video 
cap = cv2.VideoCapture('sample.mp4') 


# Loop untill the end of the video 
while (cap.isOpened()): 

	# Capture frame-by-frame 
	ret, frame = cap.read() 
	frame = cv2.resize(frame, (540, 380), fx = 0, fy = 0, 
						interpolation = cv2.INTER_CUBIC) 

	# Display the resulting frame 
	cv2.imshow('Frame', frame) 

	# conversion of BGR to grayscale is necessary to apply this operation 
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 

	# adaptive thresholding to use different threshold 
	# values on different regions of the frame. 
	Thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
										cv2.THRESH_BINARY_INV, 11, 2) 

	cv2.imshow('Thresh', Thresh) 
	# define q as the exit button 
	if cv2.waitKey(25) & 0xFF == ord('q'): 
		break

# release the video capture object 
cap.release() 
# Closes all the windows currently opened. 
cv2.destroyAllWindows() 


In [ ]:

for idx in [1]:
  print("processing video stim %d" %idx)
  frames=[]
  vid_path = "/content/gdrive/My Drive/291 Bionic Vision Project/Asa&Devi_Video/New Stimuli/Videos/stim%d_orig_frames" %idx # ORIGINAL VIDEO PATH FOLDER 
  vid_name = 'stim%d.mp4' %idx

  cap = cv2.VideoCapture('%s/%s'% (vid_path, vid_name))
  ret, frame1 = cap.read()
  #results = model.detect([frame1])
  #rs = results[0]
  # fig = visualize.display_instances(frame1, rs['rois'], rs['masks'], rs['class_ids'], 
                            # class_names, rs['scores'])


  # set up optical flow 
  prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
  hsv = np.zeros_like(frame1)
  hsv[...,1] = 255
  abc=0
  # saliency observation window
  while (cap.isOpened()): 

    # Capture frame-by-frame 
    ret, frame = cap.read() 
    #frame = cv2.resize(frame, (540, 380), fx = 0, fy = 0, 
              #interpolation = cv2.INTER_CUBIC) 

    # Display the resulting frame 
    #cv2_imshow(frame) 


  ### create processed frames and downsampled frames ###
 
    h,w,_ = frame.shape
    my_map = FasaSaliencyMapping(h, w) 

    image_salient = my_map.returnMask(frame, tot_bins=8, format='BGR2LAB')  # get the mask from the original image
    image_salient_blr = cv2.GaussianBlur(image_salient, (3,3), 1)
    #cv2_imshow(image_salient_blr)
    frames.append(image_salient_blr)
    print(abc)
    abc=abc+1

processing video stim 1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123


AttributeError: ignored

In [ ]:
os.chdir('/content/gdrive/My Drive/saliency videos')

In [ ]:

for idx in range(4,16):
  print("processing video stim %d" %idx)
  frames=[]
  vid_path = "/content/gdrive/My Drive/291 Bionic Vision Project/Asa&Devi_Video/New Stimuli/Videos/stim%d_orig_frames" %idx # ORIGINAL VIDEO PATH FOLDER 
 
    #frame = cv2.resize(frame, (540, 380), fx = 0, fy = 0, 
              #interpolation = cv2.INTER_CUBIC) 

    # Display the resulting frame 
    #cv2_imshow(frame) 
  os.chdir(vid_path)
  for j in range(125):
    fname="frame%d.jpg" %(j+1)
    ### create processed frames and downsampled frames ###
    frame=cv2.imread(fname)
    h,w,_ = frame.shape
    my_map = FasaSaliencyMapping(h, w) 

    image_salient = my_map.returnMask(frame, tot_bins=8, format='BGR2LAB')  # get the mask from the original image
    image_salient_blr = cv2.GaussianBlur(image_salient, (3,3), 1)
    #cv2_imshow(image_salient_blr)
    frames.append(image_salient_blr)
    print(j)
    

  os.chdir('/content/gdrive/My Drive/saliency videos')
  vidname="saliency_stim%d.avi" %idx
  out = cv2.VideoWriter(vidname, cv2.VideoWriter_fourcc(*'mp4v'), 20, (960, 540),isColor=False)
  for i in range(125):
    out.write(frames[i]) # frame is a numpy.ndarray with shape (1280, 720, 3)
  out.release()

processing video stim 4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
processing video stim 5
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
processing video stim 6
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52

In [ ]:
## Combining saliency and segmentation using OR and then using them with depth
for idx in [1]:
  print("processing video stim %d" %idx)
  frames=[]
  salvid_path = "/content/gdrive/My Drive/291 Bionic Vision Project/Asa&Devi_Video/New Stimuli/saliency videos/" # ORIGINAL VIDEO PATH FOLDER 
  salvid_name = 'stim%d.mp4' %idx

  segvid_path = "/content/gdrive/My Drive/291 Bionic Vision Project/Asa&Devi_Video/New Stimuli/segmentation videos/" # ORIGINAL VIDEO PATH FOLDER 
  segvid_name = 'stim%d.mp4' %idx

  depthvid_path = "/content/gdrive/My Drive/291 Bionic Vision Project/Asa&Devi_Video/New Stimuli/depth videos/" # ORIGINAL VIDEO PATH FOLDER 
  depthvid_name = 'stim%d.mp4' %idx



  capsal = cv2.VideoCapture('%s/%s'% (salvid_path, salvid_name))
  capseg = cv2.VideoCapture('%s/%s'% (segvid_path, segvid_name))
  capdepth = cv2.VideoCapture('%s/%s'% (depthvid_path, depthvid_name))
  #ret, frame1 = cap.read()
  #results = model.detect([frame1])
  #rs = results[0]
  # fig = visualize.display_instances(frame1, rs['rois'], rs['masks'], rs['class_ids'], 
                            # class_names, rs['scores'])


  # set up optical flow 
  
  abc=0
  # saliency observation window
  while (abc<=123): 

    # Capture frame-by-frame 
    retsal, framesal = capsal.read() 
    retseg, frameseg = capseg.read() 
    retdepth, framedepth = capdepth.read() 
    #frame = cv2.resize(frame, (540, 380), fx = 0, fy = 0, 
              #interpolation = cv2.INTER_CUBIC) 

    # Display the resulting frame 
    cv2_imshow(framesal) 
    print(framesal.shape)

  ### create processed frames and downsampled frames ###
    #thresh_sal=

    #frames.append(image_salient_blr)
    print(abc)
    abc=abc+1

 # os.chdir('/content/gdrive/My Drive/salsegdepth')
 # vidname="saliency_stim%d.avi" %idx
 # out = cv2.VideoWriter(vidname, cv2.VideoWriter_fourcc(*'mp4v'), 20, (960, 540),isColor=False)
 # for i in range(125):
 #   out.write(frames[i]) # frame is a numpy.ndarray with shape (1280, 720, 3)
 # out.release()